# 1. Step by step: Passing Python functions to Claude Desktop

By the end of this notebook you'll have everything you need to run Claude with a demo Python function!

We made this for ourselves & other data scientists who are more familiar with notebooks & Python to directly pass any Python code to Claude.

This notebook will:

1. Setup all you need to pass existing functions in this repo to Claude
3. Pass a function giving the time in UTC to Claude to Claude's MCP server config
4. Automatically restarting Claude with new changes

In [the next notebook we will](2.create_your_own_agents.ipynb):

1. Show you how to write your own function & save it locally
2. Create a new Agent with whichever functions you want
3. Change which (list of) Agent(s) you want Claude to know about



In [3]:
import fused
import json
import os
import time
from pathlib import Path

Update the following with your local file paths:

In [6]:
# This is the config file for MCP Servers that Claude Desktop uses
PATH_TO_CLAUDE_CONFIG = (
    f"{str(Path.home())}/Library/Application Support/Claude/claude_desktop_config.json"
)


if not os.path.exists(PATH_TO_CLAUDE_CONFIG):
    # Creating the config file
    os.makedirs(os.path.dirname(PATH_TO_CLAUDE_CONFIG), exist_ok=True)
    with open(PATH_TO_CLAUDE_CONFIG, "w") as f:
        json.dump({}, f)

assert os.path.exists(PATH_TO_CLAUDE_CONFIG), (
    "Please update the PATH_TO_CLAUDE_CONFIG variable with the correct path to your Claude config file"
)

In [7]:
# Local path to the Claude app
CLAUDE_APP_PATH = "/Applications/Claude.app"
assert os.path.exists(CLAUDE_APP_PATH), (
    "Please update the CLAUDE_APP_PATH variable with the correct path to your Claude app"
)

In [8]:
# Change this path if you're not running this from the repo root
WORKING_DIR = os.getcwd()

## Calling pre-existing Agents (that we built for you)

We've already created a few agents for you in this repo. So we'll start by showing you _how to run existing agents_ and in the [following notebook](create_your_own_agents.ipynb) we'll go over how to create your own and passing your own functions

We built a very simple "hello world" example of passing Python functions to an LLM: Getting to know the time!

This might sound like a naive example, but [current LLMs have a training cutoff date](https://github.com/HaoooWang/llm-knowledge-cutoff-dates) after which they have no knowledge of the world. This means that while it might sound simple enough, current LLM models can't know the current time. 

We're going to give Claude the ability to know the time by just running a simple Python script inside a Fused UDF that gives the current time



<!-- TODO: Explain a bit what a UDF (requirements, input / outputs) -->
<!-- Explain why time is good example for LLMs, because they have a cutoff period of time -->

In [15]:
# As MCP implementation are constantly shifting we've moved some of our implementation code to another repo to make it easier to update this notebook without major changes
commit = "5f860ab"
common = fused.load(
    f"https://github.com/fusedio/udfs/tree/{commit}/public/common"
).utils

### Our local `agents.json` file

We've build an intermediate `agents.json` file in this repo that serves as a simple inventory of which `agents` you have available for you to use. 

This JSON file contains:
- `names` -> Name of the agent. This is what we'll easily be able to change down the line to give Claude access to different functions
- `udfs` -> These are the name of the individual Fused UDFs that a given Agent can call

Think of this as different "folders" of capabilities that Claude can have, with different "files" of Python functions

In [16]:
# Checking the agents.json file
agents = json.load(open(os.path.join(WORKING_DIR, "agents.json")))
print(json.dumps(agents, indent=4, sort_keys=True))

{
    "agents": [
        {
            "name": "get_current_time",
            "udfs": [
                "current_utc_time"
            ]
        },
        {
            "name": "fused_docs",
            "udfs": [
                "list_public_udfs",
                "reading_fused_docs"
            ]
        },
        {
            "name": "getting_the_news",
            "udfs": [
                "recent_hacker_news_stories"
            ]
        }
    ]
}


### Generating `claude_desktop_config.json`

Claude Desktop App is expecting a specific JSON file that tells it: what to run, and where. 

The following helper function simply edits this `claude_desktop_config.json` under the hood for us and tells Claude which Agent we want to run from the `agents.json` above

For this simple example, we'll use the pre-existing `get_current_time` Agent (which runs the `current_utc_time` UDF as per the `agents.json` file above)

In [17]:
common.generate_local_mcp_config(
    config_path=PATH_TO_CLAUDE_CONFIG,
    agents_list=["get_current_time"],
    repo_path=WORKING_DIR,
)

In [18]:
# Reading claude config to confirm
json.load(open(PATH_TO_CLAUDE_CONFIG))

{'mcpServers': {'get_current_time': {'command': 'uv',
   'args': ['run',
    '--directory',
    '/Users/maximelenormand/Library/CloudStorage/Dropbox/Mac/Documents/repos/fused-mcp',
    'main.py',
    '--runtime=local',
    '--udf-names=current_utc_time',
    '--name=get_current_time']}}}

## Now restart Claude!

Everytime we change the `claude_desktop_config.json` we need to restart Claude so we made a quick util to do it for you!

In [19]:
def restart_claude(claude_path: str = CLAUDE_APP_PATH):
    app_name = claude_path.split("/")[-1]

    try:
        os.system(f"pkill -f '{app_name}'")
        print(f"Killed {app_name}")
        time.sleep(2)  # Wait for shutdown
    except Exception:
        print("Claude wasn't running, so no need to kill it")

    print(f"Restarting {app_name}")
    os.system(f"open -a '{claude_path}'")  # Restart Claude

In [20]:
restart_claude()

Killed Claude.app
Restarting Claude.app


You might have to wait for a few seconds for Claude to load our MCP tools. Once tools are loaded it looks like this:

![Local Claude with MCP tools](img/Claude_setup_with_mcp_tools.png)

Everytime you run this you need to click "Allow For This Chat":

![Allow for each chat](img/claude_agree_for_chat.png)

## Change agents & Build your own!

We've shown you how to get started, in the next notebook [we'll show you how to write your own functions & change agent names](2.create_your_own_agents.ipynb)